# Using dGPS on GLE to Validate the Fusion Strategy

In [ ]:
# ruff: noqa: E402

%load_ext autoreload
%autoreload 2

# find the root of the project
import os
from pathlib import Path
import sys
import polars as pl

ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
sys.path.append(str(ROOT))

SHOW_FIGS = False

## Read the GLE Data


In [ ]:
# Length of the GLE

GLE_m = 4.93522
DIST_2_FRONT_m = 2.91
DIST_2_BACK_m = 1.99


veh_df = (
    pl.read_csv(
        ROOT / "data" / "vehicle_drives" / "2023-10-31.csv",
    )
    .drop("")
    .with_columns(
        pl.col("gps_time").str.strptime(
            dtype=pl.Datetime(
                time_unit="us",
            ),
        ),
    )
    .sort(
        "gps_time",
    )
    .with_row_count(name="seq")
)

## Create the Network


In [ ]:
import geopandas as gpd
from src.geometry import RoadNetwork


network = RoadNetwork(
    lane_gdf=gpd.read_file(ROOT / "data/mainline_lanes.geojson"),
    step_size=0.01,
)

lane_df = network.df

LANE_WIDTH = 3.5

## Snap the Vehicle Trajectories to Lanes


In [ ]:
import utm

x, y, _, _ = utm.from_latlon(
    latitude=veh_df["lat"].to_numpy(),
    longitude=veh_df["lon"].to_numpy(),
)

GLE_m = 4.93522
DIST_2_FRONT_m = 2.91
DIST_2_BACK_m = 1.99


veh_df = (
    veh_df.sort("gps_time")
    .with_columns(
        x=x,
        y=y,
        gps_time=pl.col("gps_time").dt.truncate("100ms"),
    )
    .with_columns(
        pl.col("x").diff().alias("dx"),
        pl.col("y").diff().alias("dy"),
    )
    .with_columns(
        (pl.col("dx").pow(2) + pl.col("dy").pow(2)).sqrt().alias("dist"),
        # find the heading
        pl.arctan2(pl.col("dy"), pl.col("dx")).alias("heading"),
    )
    .with_columns(
        pl.col("dist").cum_sum().alias("cum_dist"),
        # make the front x/y position
        (pl.col("x") + (DIST_2_FRONT_m * pl.col("heading").cos())).alias("front_x"),
        (pl.col("y") + (DIST_2_FRONT_m * pl.col("heading").sin())).alias("front_y"),
        (pl.col("x") - (DIST_2_BACK_m * pl.col("heading").cos())).alias("back_x"),
        (pl.col("y") - (DIST_2_BACK_m * pl.col("heading").sin())).alias("back_y"),
    )
    .with_columns(
        #
        (
            (pl.col("cum_dist").shift(-1) - pl.col("cum_dist").shift(1)).abs()
            / (
                (
                    pl.col("gps_time").shift(-1) - pl.col("gps_time").shift(1)
                ).dt.total_milliseconds()
                / 1e3
            )
        ).alias("speed"),
    )
    .pipe(
        network.map_to_lane,
        dist_upper_bound=LANE_WIDTH,
        utm_x_col="x",
        utm_y_col="y",
    )
    .filter(
        pl.col("name").is_not_null(),
    )
    .rename(
        {
            "name": "lane",
            "angle": "heading_lane",
        }
    )
    .with_columns((pl.col("heading") - pl.col("heading_lane")).alias("angle_diff"))
    .with_columns(
        # find the portion of velocity that is in the direction of the lane
        pl.arctan2(pl.col("angle_diff").sin(), pl.col("angle_diff").cos()).alias(
            "angle_diff"
        )
    )
    .with_columns(
        (pl.col("speed") * pl.col("angle_diff").cos()).alias("s_velocity"),
        (pl.col("speed") * pl.col("angle_diff").sin()).alias("d_velocity"),
        ((DIST_2_FRONT_m * pl.col("angle_diff").cos()) + pl.col("s")).alias("front_s"),
        (pl.col("s") - (DIST_2_BACK_m * pl.col("angle_diff").cos())).alias("back_s"),
        # do the vehicle length
        (GLE_m * pl.col("angle_diff").cos()).alias("length_s"),
    )
)

veh_df.head()

## Lable Contigous Segments on the Lanes

In [ ]:
veh_df = (
    veh_df.with_columns(
        pl.col("lane").fill_null(""),
    )
    .with_columns(
        (
            (pl.col("lane").shift(1) != pl.col("lane"))
            & (pl.col("lane").shift(1) != "")
        ).alias("sequence"),
    )
    .with_columns(
        (pl.col("sequence").cum_sum() * (pl.col("lane") != "")).alias("sequence_id"),
    )
    .filter(pl.col("sequence_id") != 0)
)

## Read in the Radar Trajectories 

Both the prcessed and unprocessed.

### Raw Radar Data

In [ ]:
from datetime import timedelta
import polars as pl
from src.radar import CalibratedRadar
from src.pipelines.open_file import prep_df
from src.pipelines.association import add_front_back_s
from src.pipelines.kalman_filter import prepare_frenet_measurement


radar_obj = CalibratedRadar(
    radar_location_path=ROOT / "configuration" / "march_calibrated.yaml",
)


raw_radar_df = (
    pl.scan_parquet(
        Path(os.environ.get("RAW_DATA_DIR")).joinpath("*.parquet"),
    )
    .with_columns(
        pl.col("epoch_time").dt.replace_time_zone("UTC"),
    )
    .with_context(veh_df.lazy())
    .filter(
        pl.col("epoch_time").is_between(
            pl.col("gps_time").min() - timedelta(minutes=1),
            pl.col("gps_time").max() + timedelta(minutes=1),
        )
    )
    .collect()
    .lazy()
    .pipe(prep_df, f=radar_obj)
    .pipe(
        network.map_to_lane,
        dist_upper_bound=8,
        utm_x_col="utm_x",
        utm_y_col="utm_y",
    )
    .filter(pl.col("name").is_not_null())
    .with_columns(
        back_x=(
            (pl.col("heading_utm").cos() * pl.col("f32_distanceToBack_m"))
            + pl.col("utm_x")
        ),
        back_y=(
            (pl.col("heading_utm").sin() * pl.col("f32_distanceToBack_m"))
            + pl.col("utm_y")
        ),
        front_x=(
            (pl.col("heading_utm").cos() * pl.col("f32_distanceToFront_m"))
            + pl.col("utm_x")
        ),
        front_y=(
            (pl.col("heading_utm").sin() * pl.col("f32_distanceToFront_m"))
            + pl.col("utm_y")
        ),
    )
    .rename(
        {
            "name": "lane",
            "angle": "heading_lane",
        }
    )
    .pipe(prepare_frenet_measurement)
    .pipe(add_front_back_s, use_median_length=False)
    .collect()
)

### Read in the Processed Radar Data

In [ ]:
processed_radar_df = pl.read_parquet(
    ROOT / "data" / "merged_october.parquet",
)

processed_radar_df.head()

## Calculate the Optimal Time Offset b/ Radar and GPS

In [ ]:
veh_df = veh_df.with_columns(
    (pl.col("lane").str.slice(-1, 1).cast(pl.UInt32) - 1).alias("lane_index"),
    pl.concat_str(pl.col("lane").str.slice(0, 3), pl.lit("1")).alias("lane"),
    pl.col("gps_time").cast(raw_radar_df["epoch_time"].dtype).alias("epoch_time"),
)

veh_df = veh_df.filter(pl.count().over("sequence_id") > 50)

### Create a Function to Calculate the Offset

In [ ]:
def offset_error(veh_df, radar_df, offset):
    # this function treis to eagerly join the two dataframes
    return (
        veh_df
        # .lazy()
        .with_columns(
            pl.col("epoch_time") + timedelta(seconds=offset),
        )
        .filter(pl.col("epoch_time").is_in(radar_df["epoch_time"].unique()))
        # .sort("s")
        .join(
            (
                radar_df
                # .sort("s").with_columns(pl.col("s").alias("s_radar"))
            ),
            # on="s",
            on=["epoch_time", "lane", "lane_index"],
            suffix="_radar",
            # strategy='nearest'
            # how="inner",
        )
        .with_columns(
            # calculate the rms error
            ((pl.col("s") - pl.col("s_radar")) ** 2).alias("s_squared_error"),
            ((pl.col("speed") - pl.col("speed_radar")) ** 2).alias(
                "s_velocity_squared_error"
            ),
        )
        .group_by(["vehicle_id", "sequence_id"])
        .agg(
            (pl.col("s_squared_error").sum() / pl.count()).sqrt().alias("s_rmse"),
            (pl.col("s_velocity_squared_error").sum() / pl.count())
            .sqrt()
            .alias("s_velocity_rmse"),
        )
        .group_by("sequence_id")
        .agg(
            pl.col("vehicle_id")
            .gather(pl.col("s_rmse").arg_min())
            .alias("min_rmse_vehicle_id"),
            pl.col("vehicle_id")
            .gather(pl.col("s_velocity_rmse").arg_min())
            .alias("min_s_velocity_rmse_vehicle_id"),
            pl.col("s_rmse").min().alias("min_rmse"),
            pl.col("s_velocity_rmse").min().alias("min_s_velocity_rmse"),
        )
        .explode(["min_rmse_vehicle_id", "min_s_velocity_rmse_vehicle_id"])
        # .collect()
    )

### Find the Optimal Offset

In [ ]:
error_summary = []

for second_offset in range(-200, 0, 1):
    error_df = offset_error(
        veh_df.with_columns(
            pl.col("gps_time")
            .cast(raw_radar_df["epoch_time"].dtype)
            .alias("epoch_time"),
            pl.col("front_s").cast(float).alias("s"),
        ),
        processed_radar_df.with_columns(
            pl.col("front_s_smooth").cast(float).alias("s"),
            pl.col("s_velocity_smooth").alias("speed_radar"),
            pl.col("lane_index").cast(pl.UInt32).alias("lane_index"),
            # pl.col('object_id').alias('vehicle_id'),
        ),
        second_offset / 10,
    )

    error_summary.append(
        error_df.with_columns(
            pl.lit(second_offset / 10).alias("second_offset"),
        )
    )

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

error_summary_df = pl.concat(error_summary)

error_summary_df_seconds = error_summary_df.group_by("second_offset").agg(
    pl.col("min_rmse").mean().alias("min_rmse"),
    pl.col("min_s_velocity_rmse").mean().alias("min_s_velocity_rmse"),
)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Scatter(
        x=error_summary_df_seconds["second_offset"],
        y=error_summary_df_seconds["min_rmse"],
        name="RMSE",
    )
)

fig.add_trace(
    go.Scatter(
        x=error_summary_df_seconds["second_offset"],
        y=error_summary_df_seconds["min_s_velocity_rmse"],
        name="Speed RMSE",
    ),
    secondary_y=True,
)

fig.update_layout(
    height=800,
    width=1200,
    xaxis_title="Second Offset",
    yaxis_title="RMSE",
    yaxis2_title="Speed RMSE",
)

### Implement the Offset and Try to Cluster the Matching Vehicles

In [ ]:
optimal_offset = error_summary_df_seconds.sort("min_rmse")["second_offset"][0]

print(f"Optimal offset: {optimal_offset}")

## Find the Vehicles that Best Match the GPS


In [ ]:
offset_error(
    veh_df.with_columns(
        pl.col("gps_time").cast(raw_radar_df["epoch_time"].dtype).alias("epoch_time"),
        pl.col("front_s").cast(float).alias("s"),
    ),
    processed_radar_df.with_columns(
        pl.col("front_s_smooth").cast(float).alias("s"),
        pl.col("s_velocity_smooth").alias("speed_radar"),
        pl.col("lane_index").cast(pl.UInt32).alias("lane_index"),
        # pl.col('object_id').alias('vehicle_id'),
    ),
    optimal_offset,
)

In [ ]:
scatter_df = (
    veh_df.with_columns(
        (pl.col("gps_time") + timedelta(seconds=optimal_offset))
        .cast(processed_radar_df["epoch_time"].dtype)
        .alias("epoch_time"),
        pl.col("front_s").cast(float).alias("s"),
    )
    .filter(pl.col("epoch_time").is_in(processed_radar_df["epoch_time"].unique()))
    .join(
        processed_radar_df.select(
            [
                pl.col("front_s_smooth").cast(float).alias("s_radar"),
                pl.col("s_velocity_smooth").alias("speed_radar"),
                pl.col("lane_index").cast(pl.UInt32).alias("lane_index"),
                "epoch_time",
                "vehicle_id",
                "lane",
            ]
        ),
        on=["epoch_time", "lane", "lane_index"],
        # how="outer",
    )
    .with_columns(
        # calculate the rms error
        ((pl.col("s") - pl.col("s_radar")) ** 2).alias("s_squared_error"),
        ((pl.col("speed") - pl.col("speed_radar")) ** 2).alias(
            "s_velocity_squared_error"
        ),
    )
    .group_by(["vehicle_id", "sequence_id"])
    .agg(
        (pl.col("s_squared_error").sum() / pl.count()).sqrt().alias("s_rmse"),
        (pl.col("s_velocity_squared_error").sum() / pl.count())
        .sqrt()
        .alias("s_velocity_rmse"),
    )
)

In [ ]:
# try to make two clusters
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=2).fit(scatter_df[["s_rmse", "s_velocity_rmse"]].to_numpy())

scatter_df = scatter_df.with_columns(
    cluster=pl.Series(dbscan.labels_).cast(str),
)

In [ ]:
import plotly.express as px


fig = px.scatter(
    scatter_df.to_pandas(),
    x="s_rmse",
    y="s_velocity_rmse",
    color="cluster",
    hover_data=["vehicle_id", "sequence_id"],
)

fig.show()

In [ ]:
good_cluster = (
    scatter_df.group_by("cluster")
    .agg(
        pl.col("s_rmse").mean().alias("s_rmse"),
    )
    .sort("s_rmse")["cluster"][0]
)

print(f"Good cluster: {good_cluster}")

In [ ]:
good_matches = scatter_df.filter(pl.col("cluster") == good_cluster)

In [ ]:
object_ids = (
    processed_radar_df.filter(
        pl.col("vehicle_id").is_in(good_matches["vehicle_id"].unique())
    )[["vehicle_id", "object_id"]]
    .unique()
    .explode("object_id")
)

In [ ]:
object_ids.head()

## Calculate the Positional Error

We filter out short matches, as this happened when the vehicle crossed the lanes during turning movements

In [ ]:
veh_df = veh_df.filter(pl.count().over("sequence_id") > 50)

In [ ]:
raw_error_df = raw_radar_df.filter(
    pl.col("object_id").is_in(object_ids["object_id"])
).join(
    object_ids,
    on="object_id",
)

# raw_error_df.head()

### Build a Paired DataFrame

In [ ]:
processed_radar_df.head()

In [ ]:
paired_df = (
    veh_df.with_columns(
        (
            pl.col("gps_time").cast(raw_error_df["epoch_time"].dtype)
            + timedelta(seconds=optimal_offset)
        ).alias("epoch_time"),
        pl.col("lane_index").cast(int),
    )
    .join(
        processed_radar_df.select(
            [
                "epoch_time",
                "front_s_smooth",
                "back_s_smooth",
                "s_velocity_smooth",
                "d_smooth",
                "d_velocity_smooth",
                "vehicle_id",
                "lane",
                "front_x",
                "front_y",
                "back_x",
                "back_y",
                pl.col("lane_index").cast(int),
            ]
        )
        .with_columns(
            ((pl.col("s_velocity_smooth") ** 2) + (pl.col("d_velocity_smooth") ** 2))
            .sqrt()
            .alias("speed_smooth"),
        )
        .filter(pl.col("vehicle_id").is_in(good_matches["vehicle_id"].unique())),
        on=["epoch_time", "lane", "lane_index"],
        suffix="_rts",
    )
    .join(
        raw_error_df.select(
            [
                "epoch_time",
                "front_s",
                "back_s",
                "s_velocity",
                "d",
                "d_velocity",
                pl.col("f32_velocityInDir_mps").alias("speed_raw"),
                pl.col("utm_x").alias("x_raw"),
                pl.col("utm_y").alias("y_raw"),
                "vehicle_id",
                "object_id",
                "ip",
            ]
        ),
        on=["epoch_time", "vehicle_id"],
        suffix="_raw",
    )
)

In [ ]:
error_df = paired_df.filter(pl.col("front_s_smooth").is_not_null())

In [ ]:
import plotly.graph_objects as go


fig = go.Figure()

error_df = error_df.sort("epoch_time")

# plot all the EB trajectories
for g, seq_df in error_df.group_by("sequence_id"):
    fig.add_trace(
        go.Scatter(
            x=seq_df["epoch_time"],
            y=seq_df["front_s"],
            mode="markers+lines",
            name=f"Sequence {g}",
            marker_color="black",
            showlegend=False,
        ),
    )

    fig.add_trace(
        go.Scatter(
            x=seq_df["epoch_time"],
            y=seq_df["front_s_smooth"],
            mode="markers+lines",
            name=f"Sequence {g}",
            marker_color="red",
            showlegend=False,
        ),
    )

    # plot the trajectories
    fig.add_trace(
        go.Scatter(
            x=seq_df["epoch_time"],
            y=seq_df["front_s_raw"],
            mode="markers",
            name=f"Sequence {g}",
            marker_color="green",
            showlegend=False,
        ),
    )


# if SHOW_FIGS:
fig.show()

In [ ]:
import numpy as np
from scipy.stats import pearsonr


def element_error(
    df: pl.DataFrame, true_col: str, pred_col: str, out_col: str = None, pearsonr_col: str = None, mae_col: str = None
) -> pl.DataFrame:
    out_col = out_col or f"{true_col}_se"
    pearsonr_col = pearsonr_col or f"{true_col}_pearsonr"
    mae_col = mae_col or f"{true_col}_mae"
    # this some f sh!t
    def lit_pearson_r(_df):
        return _df.with_columns(
            pl.lit(
                pearsonr(
                    _df[true_col].to_numpy(),
                    _df[pred_col].to_numpy(),
                )[0]
            ).alias(pearsonr_col)
        )

    return df.with_columns(
        ((pl.col(true_col) - pl.col(pred_col)) ** 2).alias(out_col),
        ((pl.col(true_col) - pl.col(pred_col)).abs()).alias(mae_col),
    ).pipe(lit_pearson_r)


# def mse(df: pl.DataFrame, true_col: str, pred_col: str) -> float:
#     return se(df, true_col, pred_col, "se")["se"].mean()


# def rmse(df: pl.DataFrame, true_col: str, pred_col: str) -> float:
#     return mse(df, true_col, pred_col) ** (1 / 2)


# def pearsonr_(df: pl.DataFrame, true_col: str, pred_col: str) -> float:
#     # not waht this is really supposed to show, as obvoiusly the two are correlated
#     return pearsonr(df[true_col].to_numpy(), df[pred_col].to_numpy())[0]

In [ ]:
smoothed_error_df = error_df.filter(
    pl.col("vehicle_id").cum_count().over(["epoch_time", "vehicle_id"]) < 1
)

smoothed_error_df = (
    smoothed_error_df.pipe(
        element_error,
        true_col="front_s",
        pred_col="front_s_smooth",
    )
    .pipe(
        element_error,
        true_col="s_velocity",
        pred_col="s_velocity_smooth",
    )
    .pipe(
        element_error,
        true_col="back_s",
        pred_col="back_s_smooth",
    )
    .pipe(
        element_error,
        true_col="speed",
        pred_col="speed_smooth",
    )
    .with_columns(
        (
            (pl.col(f"{loc}_x") - pl.col(f"{loc}_x_rts")).pow(2)
            + (pl.col(f"{loc}_y") - pl.col(f"{loc}_y_rts")).pow(2)
        ).alias(f"{loc}_xy_se")
        for loc in ["front", "back"]
    )
)


def mse_pl(col: pl.col) -> pl.Expr:
    return col.mean()


def rmse_pl(col: pl.col) -> pl.Expr:
    return mse_pl(col) ** (1 / 2)


smoothed_error_df.group_by("sequence_id").agg(
    pl.col("^.*_se$").mean().name.map(lambda x: f"{x.replace('_se', '')}_mse"),
    pl.col("^.*_se$").mean().sqrt().name.map(lambda x: f"{x.replace('_se', '')}_rmse"),
    pl.col("^.*_pearsonr$").first(),
)

In [ ]:
smoothed_error_df = error_df.filter(
    pl.col("vehicle_id").cum_count().over(["epoch_time", "vehicle_id"]) < 1
)

smoothed_error_df = (
    smoothed_error_df.pipe(
        element_error,
        true_col="front_s",
        pred_col="front_s_smooth",
    )
    .pipe(
        element_error,
        true_col="s_velocity",
        pred_col="s_velocity_smooth",
    )
    .pipe(
        element_error,
        true_col="back_s",
        pred_col="back_s_smooth",
    )
    .pipe(
        element_error,
        true_col="speed",
        pred_col="speed_smooth",
    )
    .with_columns(
        (
            (pl.col(f"{loc}_x") - pl.col(f"{loc}_x_rts")).pow(2)
            + (pl.col(f"{loc}_y") - pl.col(f"{loc}_y_rts")).pow(2)
        ).alias(f"{loc}_xy_se")
        for loc in ["front", "back"]
    )
)


def mse_pl(col: pl.col) -> pl.Expr:
    return col.mean()


def rmse_pl(col: pl.col) -> pl.Expr:
    return mse_pl(col) ** (1 / 2)


smoothed_error_df.group_by("sequence_id").agg(
    pl.col("^.*_se$").mean().name.map(lambda x: f"{x.replace('_se', '')}_mse"),
    pl.col("^.*_se$").mean().sqrt().name.map(lambda x: f"{x.replace('_se', '')}_rmse"),
    pl.col("^.*_pearsonr$").first(),
)

In [ ]:
smoothed_error_df